In [1]:
import json
from pathlib import Path

params = {}
params_path = Path("/content/params.json")
if params_path.is_file():
    with params_path.open("r", encoding="UTF-8") as params_file:
        params = json.load(params_file)

params

{'hf_dataset': 'weaviate/WithoutRetrieval-SchemaSplit-Test-80',
 'push_to_hub': 'substratusai/wgql-WithRetrieval-SchemaSplit-Train-80-3epochs'}

In [2]:
from datasets import load_dataset

hf_dataset = params.get("hf_dataset")
if hf_dataset:
    dataset = load_dataset(hf_dataset)
else:
    dataset = load_dataset("json", data_files="/content/data/*.json*")

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'nlcommand', 'apiRef', 'apiRefPath', 'schema', 'schemaPath'],
        num_rows: 825
    })
})

In [3]:
default_prompt = """
## Instruction
Your task is to write GraphQL for the Natural Language Query provided. Use the provided API reference and Schema to generate the GraphQL. The GraphQL should be valid for Weaviate.

Only use the API reference to understand the syntax of the request.

## Natural Language Query
{nlcommand}

## Schema
{schema}

## API reference
{apiRef}

## Answer
```graphql
"""

prompt = params.get("prompt_template", default_prompt)
print(prompt.format_map(dataset["train"][0]))


## Instruction
Your task is to write GraphQL for the Natural Language Query provided. Use the provided API reference and Schema to generate the GraphQL. The GraphQL should be valid for Weaviate.

Only use the API reference to understand the syntax of the request.

## Natural Language Query
```text
Show me the event name, description, year, significant impact, and the countries involved with their population for the top 10 historical events.
```

## Schema
{
"classes": [
{
"class": "HistoricalEvent",
"description": "Information about historical events",
"vectorIndexType": "hnsw",
"vectorizer": "text2vec-transformers",
"properties": [
{
"name": "eventName",
"dataType": ["text"],
"description": "Name of the historical event"
},
{
"name": "description",
"dataType": ["text"],
"description": "Detailed description of the event"
},
{
"name": "year",
"dataType": ["int"],
"description": "Year the event occurred"
},
{
"name": "hadSignificantImpact",
"dataType": ["boolean"],
"description": "Wheth

In [4]:
import transformers
import torch
import sys
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = "/content/model/"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
            model_path, load_in_8bit=True, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
! nvidia-smi

Sat Oct 14 22:30:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA L4           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    35W /  72W |   8574MiB / 23034MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
model.generation_config

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "use_cache": false
}

In [6]:
device = "cuda"
model_inputs = tokenizer([prompt.format_map(dataset["train"][0])], return_tensors="pt").to(device)


generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, top_k=50, top_p=0.95, temperature=0.3)
print(tokenizer.batch_decode(generated_ids)[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> 
## Instruction
Your task is to write GraphQL for the Natural Language Query provided. Use the provided API reference and Schema to generate the GraphQL. The GraphQL should be valid for Weaviate.

Only use the API reference to understand the syntax of the request.

## Natural Language Query
```text
Show me the event name, description, year, significant impact, and the countries involved with their population for the top 10 historical events.
```

## Schema
{
"classes": [
{
"class": "HistoricalEvent",
"description": "Information about historical events",
"vectorIndexType": "hnsw",
"vectorizer": "text2vec-transformers",
"properties": [
{
"name": "eventName",
"dataType": ["text"],
"description": "Name of the historical event"
},
{
"name": "description",
"dataType": ["text"],
"description": "Detailed description of the event"
},
{
"name": "year",
"dataType": ["int"],
"description": "Year the event occurred"
},
{
"name": "hadSignificantImpact",
"dataType": ["boolean"],
"description": "W

In [57]:
[prompt.format_map(x) for x in dataset["train"].select(range(5))]

['\n## Instruction\nYour task is to write GraphQL for the Natural Language Query provided. Use the provided API reference and Schema to generate the GraphQL. The GraphQL should be valid for Weaviate.\n\nOnly use the API reference to understand the syntax of the request.\n\n## Natural Language Query\n```text\nShow me the event name, description, year, significant impact, and the countries involved with their population for the top 10 historical events.\n```\n\n## Schema\n{\n"classes": [\n{\n"class": "HistoricalEvent",\n"description": "Information about historical events",\n"vectorIndexType": "hnsw",\n"vectorizer": "text2vec-transformers",\n"properties": [\n{\n"name": "eventName",\n"dataType": ["text"],\n"description": "Name of the historical event"\n},\n{\n"name": "description",\n"dataType": ["text"],\n"description": "Detailed description of the event"\n},\n{\n"name": "year",\n"dataType": ["int"],\n"description": "Year the event occurred"\n},\n{\n"name": "hadSignificantImpact",\n"dataTy

In [42]:
stop_ids = torch.LongTensor(tokenizer("```")["input_ids"])
## Note the stop_ids aren't correct, for some reason there are multiple possible token IDs
## so instead we're using tensor([13940, 28832], device='cuda:0') as the stop_ids
stop_ids


tensor([   1, 8789])

In [59]:
%%time
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

device = "cuda"
model_inputs = tokenizer([prompt.format_map(x) for x in dataset["train"].select(range(5))],
                         return_tensors="pt", padding=True).to(device)

class BacktickStoppingCriteria(StoppingCriteria):
    def __init__(self):
        self.stop_tokens = torch.LongTensor([13940, 28832]).to(device)
        self.len = self.stop_tokens.shape[0]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        if torch.equal(input_ids[0][-1 * self.len:], self.stop_tokens):
            return True
        return False



stopping_criteria = StoppingCriteriaList([BacktickStoppingCriteria()])

generated_ids = model.generate(**model_inputs,
                               max_new_tokens=300,
                               stopping_criteria=stopping_criteria)

print(tokenizer.batch_decode(generated_ids)[1])

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[PAD][PAD][PAD][PAD][PAD]<s> 
## Instruction
Your task is to write GraphQL for the Natural Language Query provided. Use the provided API reference and Schema to generate the GraphQL. The GraphQL should be valid for Weaviate.

Only use the API reference to understand the syntax of the request.

## Natural Language Query
```text
Show me the name, description, year introduced, and whether it is a string of 10 instruments. Also, show me the name, genre, and years active of the musicians who play those instruments.
```

## Schema
{
"classes": [
{
"class": "Instrument",
"description": "A musical instrument.",
"vectorIndexType": "hnsw",
"vectorizer": "text2vec-transformers",
"properties": [
{
"name": "name",
"dataType": ["text"],
"description": "Name of the instrument."
},
{
"name": "description",
"dataType": ["text"],
"description": "Description of the instrument."
},
{
"name": "yearIntroduced",
"dataType": ["int"],
"description": "Year the instrument was introduced."
},
{
"name": "isString"